# 🧠 Hotdog Classifier (ResNet18 with PyFunc Logging)

In [1]:
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import mlflow
import mlflow.pytorch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
BATCH_SIZE = 32
EPOCHS = 25
NUM_CLASSES = 2
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "data"


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "train"), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(DATA_DIR, "test"), transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [4]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE).float()


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5)
PATIENCE = 5
best_val_acc = 0
patience_counter = 0


In [6]:
def evaluate(model, dataloader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(DEVICE).float(), y.to(DEVICE)
            outputs = model(x)
            _, preds = torch.max(outputs, 1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    return y_true, y_pred


In [7]:
train_losses, train_accuracies = [], []

for epoch in range(EPOCHS):
    model.train()
    running_loss, correct = 0.0, 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE).float(), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    train_loss = running_loss / len(train_loader.dataset)
    train_acc = correct / len(train_loader.dataset)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    y_true, y_pred = evaluate(model, test_loader)
    val_acc = accuracy_score(y_true, y_pred)
    scheduler.step(val_acc)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pt")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping triggered.")
            break


Epoch 1, Train Loss: 0.4844, Train Acc: 0.7751, Val Acc: 0.8520
Epoch 2, Train Loss: 0.2014, Train Acc: 0.9277, Val Acc: 0.8640
Epoch 3, Train Loss: 0.1090, Train Acc: 0.9659, Val Acc: 0.8760
Epoch 4, Train Loss: 0.0886, Train Acc: 0.9759, Val Acc: 0.8540
Epoch 5, Train Loss: 0.0476, Train Acc: 0.9880, Val Acc: 0.8680
Epoch 6, Train Loss: 0.0176, Train Acc: 0.9980, Val Acc: 0.8860
Epoch 7, Train Loss: 0.0423, Train Acc: 0.9940, Val Acc: 0.8740
Epoch 8, Train Loss: 0.0186, Train Acc: 0.9980, Val Acc: 0.8760
Epoch 9, Train Loss: 0.0194, Train Acc: 0.9980, Val Acc: 0.9000
Epoch 10, Train Loss: 0.0395, Train Acc: 0.9900, Val Acc: 0.8720
Epoch 11, Train Loss: 0.0197, Train Acc: 0.9940, Val Acc: 0.8920
Epoch 12, Train Loss: 0.0173, Train Acc: 0.9940, Val Acc: 0.8800
Epoch 13, Train Loss: 0.0105, Train Acc: 1.0000, Val Acc: 0.9040
Epoch 14, Train Loss: 0.0105, Train Acc: 0.9980, Val Acc: 0.9020
Epoch 15, Train Loss: 0.0149, Train Acc: 0.9940, Val Acc: 0.9040
Epoch 16, Train Loss: 0.0167, Trai

In [8]:
import mlflow.pyfunc

class WrappedResNet18(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        import torch
        import torch.nn as nn
        import torchvision.models as models
        weights_path = context.artifacts["model_weights"]
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, 2)
        model.load_state_dict(torch.load(weights_path, map_location="cpu"))
        model.eval()
        self.model = model

    def predict(self, context, model_input):
        import torch
        import numpy as np
        arr = np.array(model_input, dtype=np.float32)
        tensor = torch.tensor(arr).view(-1, 3, 224, 224)
        with torch.no_grad():
            output = self.model(tensor)
            return torch.argmax(output, dim=1).numpy()


C:\Users\ahsan\onedrive\documents\mlops\hotdog-classifier\venv\lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


In [9]:
torch.save(model.state_dict(), "best_model.pt")

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("hotdog-classifier-resnet18-v3")

with mlflow.start_run():
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=WrappedResNet18(),
        artifacts={"model_weights": "best_model.pt"}
    )
    print("✅ PyFunc model logged and ready to serve.")


2025/06/15 22:16:36 INFO mlflow.tracking.fluent: Experiment with name 'hotdog-classifier-resnet18-v3' does not exist. Creating a new experiment.
2025/06/15 22:16:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ PyFunc model logged and ready to serve.
🏃 View run shivering-mouse-427 at: http://127.0.0.1:5000/#/experiments/771635954034062219/runs/1e69b88971fb42cc8fea36a919395ef0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/771635954034062219
